In [5]:
# Giải điều chế âm thanh nhận được
import warnings
import numpy as np
from scipy import signal
from scipy.io.wavfile import read

def decode(audio_signal, sample_rate=44100, baud_rate=50):
    samples_per_bit = sample_rate // baud_rate
    num_bits = len(audio_signal) // samples_per_bit
    binary_string = ''
    for i in range(num_bits):
        bit_signal = audio_signal[i*samples_per_bit:(i+1)*samples_per_bit]
        f, Pxx = signal.periodogram(bit_signal, sample_rate)
        
        if f[np.argmax(Pxx)] > 10000:
            binary_string += '1'
        else:
            binary_string += '0'
    return binary_string


with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    sample_rate, audio_signal = read('rec.wav')

# Sau khi giải mã file ghi âm ta sẽ nhận được dòng bit có các bit 0 dư thừa ở đầu và cuối tín hiệu
decoded_binary_string = decode(audio_signal, sample_rate)
print(decoded_binary_string)

001100110111010110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000101010011000101101000001100010110010100110010011011000011000101101111000010100101011111011011001010001010100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [6]:
# Xoá bỏ chuỗi tiền tố, hậu tố, các bit 0 dư thừa
def remove_extra_bits(binary_string, preamble='010101', postamble='101010'):
    start_index = binary_string.find(preamble) + len(preamble)
    end_index = binary_string.rfind(postamble)
# Nếu không tìm thấy tiền tố và hậu tố, thông báo "Tín hiệu nhận bị lỗi"
    if start_index == -1 or end_index == -1:
        print('Tín hiệu nhận bị lỗi')
        return
    return binary_string[start_index:end_index]

# Định nghĩa chuỗi tiền tố và hậu tố
preamble = '010101'
postamble = '101010'
 
# Giải mã và in kết quả
CRC_binary_string = remove_extra_bits(decoded_binary_string, preamble, postamble)
if CRC_binary_string:
    print(CRC_binary_string)

001100010110100000110001011001010011001001101100001100010110111100001010010101111101101100101000


In [7]:
# Giải mã kênh mã hoá bằng mã phát hiện lỗi CRC
import binascii

def crc_decode(CRC_binary_string):
    # Tách chuỗi nhị phân thành phần dữ liệu và phần CRC
    data_binary = CRC_binary_string[:-32]
    crc_binary = CRC_binary_string[-32:]

    # Chuyển đổi phần dữ liệu từ nhị phân sang bytes
    data = int(data_binary, 2).to_bytes((len(data_binary) + 7) // 8, byteorder='big')

    # Tính toán CRC của phần dữ liệu
    crc = binascii.crc32(data)
    crc_calculated_binary = format(crc, '032b')

    # Kiểm tra xem CRC được tính toán có khớp với phần CRC từ đầu vào hay không
    if crc_calculated_binary == crc_binary:
        return data_binary
    else:
        return None

# Giải mã
binary_string = crc_decode(CRC_binary_string)

# In ra
if binary_string is not None:
    print(binary_string)
else:
    print('Tín hiệu nhận bị lỗi')

0011000101101000001100010110010100110010011011000011000101101111


In [8]:
# Giải mã nguồn được mã hoá RLE về văn bản ban đầu

def run_length_decoding(binary_string):
    if not binary_string:
        print('Tín hiệu nhận bị lỗi')
        return
    decoded_text = ""
    n = int(len(binary_string)/8)
    encoded_text = ''.join(chr(int(binary_string[i*8:i*8+8], 2)) for i in range(n))
    i = 0
    while i < len(encoded_text):
        count = int(encoded_text[i])
        char = encoded_text[i + 1]
        decoded_text += char * count
        i += 2
    return decoded_text

decoded_text = run_length_decoding(binary_string)

# In ra các văn bản nhận được và lưu vào file decoded_text.txt
if decoded_text:
    print(decoded_text)
file_name = 'decoded_text.txt'
with open(file_name, 'w') as f:
    f.write(decoded_text)

hello
